---
title: "Final Project (Group 14)"
format: html
---


# Import Packages

In [ ]:
import pandas as pd
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import altair as alt
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
from textblob import TextBlob
import spacy

# Import Illegal Pets Data and Education Attainment Data in NYC

In [ ]:
# Illegal Pets Data
illegal_pets = pd.read_csv("data/Illegal Pets/illegal_animals_kept_as_pets_20241116.csv")

# Education Attainment Data
education = pd.read_csv("data/Education Attainment/ACSST5Y2010.S1501-Data.csv")

# Combine all education data from 2010 to 2022

In [ ]:
# Directory where all CSV files are stored
directory = "data/Education Attainment"

# Collect all file paths in the directory that match the naming pattern
csv_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".csv")]

# List to hold individual DataFrames
dataframes = []

# Loop through each file, read it into a DataFrame, and append it to the list
for file in csv_files:
    # Extract the year from the file name (e.g., "ACSST5Y2010" -> "2010")
    year = os.path.basename(file).split("ACSST5Y")[1][:4]
    df = pd.read_csv(file)
    df['YEAR'] = year
    
    # Append the modified DataFrame to the list
    dataframes.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

# Sort the combined DataFrame by 'GEO_ID' in descending order
sorted_df = combined_df.sort_values(by='GEO_ID', ascending=False)

# Anayze meanings of variables
Each year's dataset's variable has different interpretation, so it it critical to firstly select the most relevant variable for futher analysis. In our project, we only select total estimate to simplify our anlysis without considering gender and earning. For example, S1501_C01_015's definition has changed in 2018 from 'percent of bachelor degree or higher' to 'total population of bachelor degreee or higher.' Thus, we have to manually clean these messy variables.

In [ ]:
# Select the top 13 rows (containing each years variable definition) to analyze vairbales meaning
top_13_rows = sorted_df.head(13)

# Identify columns where any cell contains "female", "male", or "margin"
columns_to_drop = top_13_rows.columns[
    top_13_rows.apply(lambda col: col.astype(str).str.contains(r"female|male|margin|nan|earning|race", case=False, na=False).any())
]

# Drop the identified columns
top_13_rows_f = top_13_rows.drop(columns=columns_to_drop)
top_13_rows_f = top_13_rows_f.dropna(axis=1, how='all')

# Select necessery variables

In [ ]:
# Cleaning Education Attainment Data
# Rename selected columns for better readability
column_rename_map = {
    "GEO_ID": "geo_id",  # Geographic identifier
    "NAME": "area_name",  # Geographic area name
    "YEAR": "year",  # Year of data
    "S1501_C01_006E": "pop_25_plus",  # Population 25 years and over
    "S1501_C01_007E": "pop_25_less_9th",  # 25 years and over: Less than 9th grade
    "S1501_C01_009E": "pop_25_hs_grad",  # 25 years and over: High school graduate
    "S1501_C01_013E": "pop_25_bach_plus",  # 25 years and over: Bachelor's degree or higher
    "S1501_C01_019E": "pop_25_34_bach_plus",  # 25-34: Bachelor's degree or higher
    "S1501_C01_021E": "pop_45_64_bach_plus"  # 45-64: Bachelor's degree or higher
}

# Create selected_columns based on column_rename_map keys
selected_columns = list(column_rename_map.keys())

# Filter the combined DataFrame to only keep the selected columns
filtered_df = combined_df[selected_columns]

# Rename the columns
filtered_df.rename(columns=column_rename_map, inplace=True)

# Remove rows where GEO_ID equals "Geographic Area Name"
edu_cleaned = filtered_df[filtered_df["geo_id"] != "Geographic Area Name"]

# Adjust variables to accommodate different years' approaches to estimate (volume or percentage)
# Transform year 2010 to year 2014's 'pop_25_less_9th' variable to be the result between 'pop_25_less_9th' and 'pop_25_plus'
for year in range(2010, 2015):
    edu_cleaned.loc[edu_cleaned['year'] == str(year), 'pop_25_less_9th'] = pd.to_numeric(edu_cleaned['pop_25_less_9th'], errors='coerce') * pd.to_numeric(edu_cleaned['pop_25_plus'], errors='coerce') / 100
    edu_cleaned.loc[edu_cleaned['year'] == str(year), 'pop_25_hs_grad'] = pd.to_numeric(edu_cleaned['pop_25_hs_grad'], errors='coerce') * pd.to_numeric(edu_cleaned['pop_25_plus'], errors='coerce') / 100
    edu_cleaned.loc[edu_cleaned['year'] == str(year), 'pop_25_bach_plus'] = pd.to_numeric(edu_cleaned['pop_25_bach_plus'], errors='coerce') * pd.to_numeric(edu_cleaned['pop_25_plus'], errors='coerce') / 100

# Cleaning Illegal Pets Data
# List of columns to drop
columns_to_drop = [
    'Agency', 'Agency Name', 'Complaint Type', 'Cross Street 1', 'Cross Street 2',
    'Intersection Street 1', 'Intersection Street 2', 'City', 'Landmark',
    'Facility Type', 'Community Board', 'Park Facility Name', 'Vehicle Type',
    'Taxi Company Borough', 'Taxi Pick Up Location', 'Bridge Highway Name',
    'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment'
]

# Drop the specified columns
illegal_pets = illegal_pets.drop(columns=columns_to_drop, errors='ignore')

# Standardization of County Name

In [ ]:
# Exam unqiue values in both dataframes for furthern merging
print(edu_cleaned['area_name'].unique())

# Create county name list in NYC
nyc_counties = [
    "New York County, New York",  # Manhattan
    "Kings County, New York",     # Brooklyn
    "Queens County, New York",    # Queens
    "Bronx County, New York",     # The Bronx
    "Richmond County, New York"   # Staten Island
]

# Select counties in NYC and filter them
edu_nyc = edu_cleaned[edu_cleaned['area_name'].isin(nyc_counties)]

# Mapping of county names to simplified borough names
county_replacement_map = {
    "New York County, New York": "New York",
    "Kings County, New York": "Kings",
    "Queens County, New York": "Queens",
    "Bronx County, New York": "The Bronx",
    "Richmond County, New York": "Staten Island"
}

# Replace county names in the 'name' column using the map
edu_nyc['area_name'] = edu_nyc['area_name'].replace(county_replacement_map)

# Exam unqiue values in both dataframes for furthern merging
print(illegal_pets['Borough'].unique())

# Mapping Borough
borough_to_county = {
    "BROOKLYN": "Kings",
    "STATEN ISLAND": "Staten Island",
    "QUEENS": "Queens",
    "MANHATTAN": "New York",
    "BRONX": "The Bronx"
}

# Map the 'Borough' column to new 'County' values
illegal_pets['area_name'] = illegal_pets['Borough'].map(borough_to_county)

print(illegal_pets['area_name'].unique())
print(edu_nyc['area_name'].unique())

# Standardization Time Variable

In [ ]:
# Format both columns as strings
illegal_pets['year'] = pd.to_datetime(illegal_pets['Created Date'], errors='coerce').dt.strftime('%Y')
edu_nyc['year'] = pd.to_datetime(edu_nyc['year'], errors='coerce').dt.strftime('%Y')

# Merging 

In [ ]:
# Drop 2023 and 2024 because education data does not includes these years
illegal_pets = illegal_pets[~illegal_pets['year'].isin(['2023', '2024'])]

# Merge based on year and area_name (County name)
merged_df = pd.merge(illegal_pets, edu_nyc, on=['year', 'area_name'], how='left')
print(merged_df.head())

# Save the merged DataFrame to a CSV file
merged_df.to_csv("final_education_illegal_pets.csv", index=False)

# Load Map Data, drop NA and convert data type

In [ ]:
# Load the GeoJSON file
geojson_path = 'nyc-zips.geojson'
nyc_zips = gpd.read_file(geojson_path)

# Drop rows with NaN ZIP codes or fill them with a placeholder
merged_df = merged_df.dropna(subset=['Incident Zip'])

# Convert ZIP codes to strings without decimals
merged_df['Incident Zip'] = merged_df['Incident Zip'].apply(lambda x: str(int(x)))

# Ensure both columns are strings
nyc_zips['postalCode'] = nyc_zips['postalCode'].astype(str)

Final Project Visualization

# 1. Bar chart: types of illegal pets in NYC

In [ ]:
# 1. Bar chart of types of illegal pets
chart = alt.Chart(merged_df).mark_bar().encode(
    y=alt.Y('Descriptor:N', 
            sort='-x',  # Sort by count in descending order
            title='Pet Type'),
    x=alt.X('count():Q',
            title='Number of Incidents'),
    tooltip=['Descriptor', alt.Tooltip('count():Q', title='Count')]
).properties(
    title='Types of Illegal Pets in NYC',
    width=500,
    height=400
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_title(
    fontSize=16,
    anchor='middle'
)

chart
chart.save("pictures/bar_chart_types_of_illegal_pets.png")

# 2. Pie chart of incidents across boroughs

In [ ]:
# Calculate percentages and create pie chart with labels
total = len(merged_df)
df_with_pct = merged_df.assign(
    percentage=lambda x: (x.groupby('Borough')['Borough'].transform('count') / total * 100).round(1)
)

# Create a summary dataframe for the text labels
summary_df = (df_with_pct.groupby('Borough')
             .agg(count=('Borough', 'count'),
                  percentage=('percentage', 'first'))
             .reset_index()
             .assign(label=lambda x: x['Borough'] + ': ' + x['percentage'].astype(str) + '%'))

# Create pie chart with percentages
chart = alt.Chart(df_with_pct).mark_arc(outerRadius=180).encode(
    theta='count():Q',
    color=alt.Color('Borough:N', 
                   scale=alt.Scale(scheme='tableau10'),
                   legend=alt.Legend(title="Borough")),
    tooltip=['Borough:N', 
            alt.Tooltip('count():Q', title='Count'),
            alt.Tooltip('percentage:Q', title='Percentage', format='.1f')]
).properties(
    title={
        'text': 'Distribution of Illegal Pet Incidents by Borough',
        'fontSize': 16,
        'anchor': 'middle'
    },
    width=500,
    height=400
)

# Add text labels with percentages
text = alt.Chart(summary_df).mark_text(radius=120, size=11).encode(
    theta=alt.Theta('count:Q', stack=True),
    text='label:N'
)

# Combine chart and labels
final_chart = (chart + text).configure_view(
    strokeWidth=0
)

final_chart
final_chart.save("pictures/pie_chart_incidents_by_borough.png")

# 3. Choropleth Map

In [ ]:
import geopandas as gpd
import pandas as pd
import json

# Load GeoJSON file
geojson_path = 'nyc-zips.geojson'
nyc_zips = gpd.read_file(geojson_path)
merged_df = pd.read_csv("final_education_illegal_pets.csv")

# Data Preprocessing
merged_df = merged_df.dropna(subset=['Incident Zip'])
merged_df['Incident Zip'] = merged_df['Incident Zip'].apply(lambda x: str(int(x)))
nyc_zips['postalCode'] = nyc_zips['postalCode'].astype(str)

# Count complaints by ZIP code
pet_counts = merged_df.groupby('Incident Zip').size().reset_index(name='complaints_count')

# Merge counts with GeoDataFrame
nyc_zips = nyc_zips.merge(pet_counts, left_on='postalCode', right_on='Incident Zip', how='left')
nyc_zips['complaints_count'] = nyc_zips['complaints_count'].fillna(0)

# Convert GeoJSON to TopoJSON (Altair supports TopoJSON)
geojson_data = json.loads(nyc_zips.to_json())

# Create Altair Chart
choropleth = alt.Chart(alt.Data(values=geojson_data['features'])).mark_geoshape().encode(
    color=alt.Color('properties.complaints_count:Q', 
                    scale=alt.Scale(scheme='orangered'), 
                    title='Illegal Pet Complaints'),
    tooltip=[
        alt.Tooltip('properties.postalCode:N', title='ZIP Code'),
        alt.Tooltip('properties.complaints_count:Q', title='Complaints')
    ]
).transform_calculate(
    complaints_count='datum.properties.complaints_count'
).properties(
    title='Choropleth Map of Illegal Pet Complaints in NYC by ZIP Code',
    width=500,
    height=400
)

# Display the chart
choropleth
choropleth.save("pictures/choropleth_map_illegal_pets.png")

# 4. Line Plot: Different education level over time

In [ ]:
import altair as alt

# Prepare the data (keeping the same data preparation steps)
education_levels_over_time = merged_df.groupby('year')[
    ['pop_25_less_9th', 'pop_25_hs_grad', 'pop_25_bach_plus']
].mean().reset_index()

education_levels_long = education_levels_over_time.melt(
    id_vars='year',
    value_vars=['pop_25_less_9th', 'pop_25_hs_grad', 'pop_25_bach_plus'],
    var_name='Education Level',
    value_name='Population'
)

education_level_mapping = {
    'pop_25_less_9th': 'Less than 9th Grade',
    'pop_25_hs_grad': 'High School Graduate',
    'pop_25_bach_plus': 'Bachelor\'s Degree or Higher'
}
education_levels_long['Education Level'] = education_levels_long['Education Level'].map(education_level_mapping)

# Create the line chart using Altair
chart = alt.Chart(education_levels_long).mark_line(point=True).encode(
    x=alt.X('year:O', title='Year'),
    y=alt.Y('Population:Q', title='Average Population'),
    color=alt.Color('Education Level:N', title='Education Level'),
    tooltip=['year', 'Education Level', alt.Tooltip('Population:Q', format=',')]
).properties(
    title='Different Education Levels Over Time',
    width=500,
    height=400
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_title(
    fontSize=16
)

chart

# 5. Bar Plot: Different education level by districts

In [ ]:
import altair as alt
import pandas as pd

# Aggregate education data by borough
education_levels_by_district = merged_df.groupby('Borough')[
    ['pop_25_less_9th', 'pop_25_hs_grad', 'pop_25_bach_plus']
].mean().reset_index()

# Melt the data for easier plotting with Altair
education_levels_long = education_levels_by_district.melt(
    id_vars='Borough',
    value_vars=['pop_25_less_9th', 'pop_25_hs_grad', 'pop_25_bach_plus'],
    var_name='Education Level',
    value_name='Population'
)

# Map the correct labels to the "Education Level" column
education_level_mapping = {
    'pop_25_less_9th': 'Less than 9th Grade',
    'pop_25_hs_grad': 'High School Graduate',
    'pop_25_bach_plus': 'Bachelor\'s Degree or Higher'
}
education_levels_long['Education Level'] = education_levels_long['Education Level'].map(education_level_mapping)

 
# Create Altair chart
chart = alt.Chart(education_levels_long).mark_bar().encode(
    x=alt.X('Borough:N', title='Borough'),
    y=alt.Y('Population:Q', title='Average Population'),
    color=alt.Color('Education Level:N', title='Education Level'),
    tooltip=['Borough', 'Education Level', 'Population']
).properties(
    title='Different Education Levels by District',
    width=500,
    height=400
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_title(
    fontSize=16,
    anchor='middle'
)

chart

# 6. Correlation Analysis


In [ ]:
import requests
import geopandas as gpd
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import folium

# 2. Aggregate Data
# Count illegal pet incidents by district
incident_counts = merged_df.groupby('Borough')['Descriptor'].count().reset_index()
incident_counts.rename(columns={'Descriptor': 'Illegal_Pet_Incidents'}, inplace=True)

# Calculate mean education levels by district
education_data = merged_df.groupby('Borough')[
    ['pop_25_less_9th', 'pop_25_hs_grad', 'pop_25_bach_plus', 'pop_25_plus']
].mean().reset_index()

# Merge datasets
district_data = pd.merge(incident_counts, education_data, on='Borough')

# Calculate the percentage of each education level relative to the total population
district_data['pop_25_less_9th_percentage'] = (district_data['pop_25_less_9th'] / district_data['pop_25_plus']) * 100
district_data['pop_25_hs_grad_percentage'] = (district_data['pop_25_hs_grad'] / district_data['pop_25_plus']) * 100
district_data['pop_25_bach_plus_percentage'] = (district_data['pop_25_bach_plus'] / district_data['pop_25_plus']) * 100

# Calculate incident percentage
district_data['Incident_Percentage(%)'] = (district_data['Illegal_Pet_Incidents'] / district_data['pop_25_plus']) * 100

# 3. Perform Regression: Using all education levels (as percentages) as predictors
X = district_data[['pop_25_less_9th_percentage', 'pop_25_hs_grad_percentage', 'pop_25_bach_plus_percentage']]
y = district_data['Incident_Percentage(%)']

# Add a constant to the model (for the intercept)
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(y, X).fit()
print(model.summary())

import altair as alt
import pandas as pd

# Use Altair to create regression plots for each feature
for feature, label in zip(
    ['pop_25_less_9th_percentage', 'pop_25_hs_grad_percentage', 'pop_25_bach_plus_percentage'],
    ["% Population with Less than 9th Grade Education",
     "% Population with High School Graduation",
     "% Population with Bachelor's Degree or Higher"]
):
    # Determine the min and max for the x-axis range with a little padding
    x_min = district_data[feature].min() * 0.9  # 10% padding below min
    x_max = district_data[feature].max() * 1.1  # 10% padding above max

    # Create Altair chart
    chart = alt.Chart(district_data).mark_point(filled=True, size=100).encode(
        x=alt.X(
            f'{feature}:Q', 
            title=label, 
            scale=alt.Scale(domain=[x_min, x_max])  # Dynamically set x-axis range
        ),
        y=alt.Y('Incident_Percentage(%):Q', title="Illegal Pet Incident Percentage (%)"),
        tooltip=[
            'Borough', 
            alt.Tooltip(f'{feature}:Q', title=label), 
            alt.Tooltip('Incident_Percentage(%):Q', title='Incident %')
        ]
    )

    # Add district names as text annotations
    text_labels = alt.Chart(district_data).mark_text(
        align='left',  
        dx=7,   
        fontSize=8   
    ).encode(
        x=alt.X(f'{feature}:Q'),
        y=alt.Y('Incident_Percentage(%):Q'),
        text='Borough'  
    )

    # Add regression line
    regression_line = chart.transform_regression(
        f'{feature}', 'Incident_Percentage(%)'
    ).mark_line(color='red')

    # Combine scatter plot and regression line
    combined_chart = chart + regression_line + text_labels

    # Display the chart directly
    display(combined_chart.properties(
        title=f"Regression: Illegal Pet Incident Percentage vs. {label}",
        width=800,
        height=400
    ).configure_title(
        fontSize=10,
        anchor='middle'
    ).configure_axis(
        labelFontSize=8,
        titleFontSize=10
    ))
    combined_chart.save(f"pictures/regression_plot_{feature}.png")

# Why Use This Method
This method normalizes incident counts by population size, allowing fair comparisons across boroughs. Regression analysis helps identify relationships between education levels as percentages and incident rates, while visualization makes these patterns clear.

By normalizing incident counts per 10,000 people, the method ensures fair comparisons across boroughs with different population sizes. Regression analysis quantifies the relationship between education levels and incident rates, providing insights into potential trends. Visualizations, such as scatter plots, make these relationships clear and accessible, helping to identify patterns and inform decision-making.

# Analysis of the Graphs
The regression coefficients are very small, reflecting the rare nature of illegal pet ownership (0.2% to 1.0% of population). Less than 9th-grade education shows a negative coefficient (-0.0466), high school graduation shows a slight negative relationship (-0.0335), and bachelor's degree shows a negative relationship (-0.0400).

The graphs appear to show strong negative relationships primarily due to scaling effects. While the x-axis spans a large range (showing education levels in thousands), the y-axis variation is minimal (0.2% to 1.0% incident rate). This disparity in scales makes even small changes appear more dramatic visually. For instance, the positive coefficient for high school graduation (3.576e-06) appears negative in the graph because the effect is so small relative to the axis scales. These patterns, while statistically subtle, suggest that education levels have a minor influence on illegal pet ownership in NYC, though other factors likely play important roles in these relationships.

# Limitations
The analysis is limited by the small sample size, with only five boroughs, which reduces the statistical power and makes it difficult to draw definitive conclusions. The use of borough-level aggregation may mask important variations within boroughs, potentially overlooking localized trends.  Additionally, while the analysis identifies correlations between education levels and incident rates, it cannot establish causation. 

# Conclusion
Despite its limitations, this method offers a useful framework for understanding the relationship between educational attainment and illegal pet incidents in NYC. By normalizing data and employing regression analysis, the study provides insights that can inform policy decisions and highlight areas for further research. The visualizations effectively communicate complex data, making it accessible to a wider audience and supporting informed decision-making.


In [ ]:
# 1. Download NYC borough boundaries GeoJSON
url = "https://data.cityofnewyork.us/api/geospatial/7t3b-ywvw?method=export&format=GeoJSON"
response = requests.get(url)

# Save the GeoJSON to a file
geojson_path = "nyc_boroughs.geojson"
with open(geojson_path, "wb") as file:
    file.write(response.content)

# Load the GeoJSON data
boroughs_gdf = gpd.read_file(geojson_path)
# Merge regression data with GeoJSON
map_data = boroughs_gdf.merge(district_data, left_on='boro_name', right_on='Borough', how='left')

# Create the map
m = folium.Map(location=[40.7128, -74.0060], zoom_start=10)

# Add choropleth for regression residuals
map_data['Residuals'] = model.resid  
folium.Choropleth(
    geo_data=map_data,
    data=map_data,
    columns=['boro_name', 'Residuals'],
    key_on='feature.properties.boro_name',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Regression Residuals",
).add_to(m)

# Add district names as labels
for _, row in map_data.iterrows():
    folium.Marker(
        location=[row.geometry.centroid.y, row.geometry.centroid.x],  # Use centroid for label position
        popup=f"Borough: {row['boro_name']}\nResidual: {row['Residuals']:.2f}",
        icon=folium.DivIcon(html=f"<div style='font-size: 12px; color: black;'>{row['boro_name']}</div>")
    ).add_to(m)

# Save and display the map
m.save("regression_map_with_labels.html")
print("Map saved as 'regression_map_with_labels.html'. Open this file to view it.")

The choropleth map of regression residuals reveals varying model performance across NYC boroughs, with the Bronx showing higher actual incident rates than predicted (red), Manhattan aligning closely with predictions (light orange), and Queens/Brooklyn showing moderate deviations (orange). These patterns suggest that while our education-based regression model captures some patterns in illegal pet ownership, it may not account for other important factors affecting incident rates, particularly in the Bronx. Limitations of this visualization include potential oversimplification of spatial patterns due to borough-level aggregation, which masks within-borough variations, and the challenge of interpreting residuals without considering the underlying population density or socioeconomic factors beyond education that might influence illegal pet ownership patterns.


In [ ]:
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import altair as alt

def analyze_resolutions_nlp():
    # Simple preprocessing without NLTK
    def preprocess_text(text):
        if pd.isna(text):
            return ""
        return str(text).lower()
    
    # Add sentiment analysis
    def get_sentiment(text):
        if pd.isna(text):
            return 0
        blob = TextBlob(str(text))
        return blob.sentiment.polarity
    
    # Create preprocessed text column and sentiment scores
    merged_df['processed_resolution'] = merged_df['Resolution Description'].apply(preprocess_text)
    merged_df['sentiment_score'] = merged_df['Resolution Description'].apply(get_sentiment)
    
    # Rest of your existing category templates and TF-IDF code...
    category_templates = {
        'Warning Issued': 'warning documentation owner landlord violation instruct correct',
        'Access Denied': 'could not access unable to gain access site inspect property',
        'No Violations Found': 'no violations found no violations cited inspection conducted',
        'Violations Found': 'notice of violation violations rules regulations issued',
        'Referred Other Agency': 'not within jurisdiction referred to another agency',
        'Under Investigation': 'will review in process investigating inspection',
        'Invalid Information': 'could not locate incorrect address incomplete information',
        'Case Closed': 'reviewed and closed request complete'
    }
    
    vectorizer = TfidfVectorizer(lowercase=True, stop_words='english', max_features=1000)
    template_vectors = vectorizer.fit_transform(category_templates.values())
    description_vectors = vectorizer.transform(merged_df['processed_resolution'])
    
    def get_category(vector):
        if vector.nnz == 0:
            return "No Resolution Recorded"
        similarities = cosine_similarity(vector, template_vectors)
        best_match_idx = similarities.argmax()
        if similarities[0][best_match_idx] < 0.1:
            return "Other Resolution"
        return list(category_templates.keys())[best_match_idx]
    
    # Apply categorization
    merged_df['resolution_category'] = [get_category(vector) for vector in description_vectors]
    
    # Create category statistics
    category_stats = merged_df['resolution_category'].value_counts()
    
    # Calculate average sentiment by category
    sentiment_by_category = merged_df.groupby('resolution_category')['sentiment_score'].agg(['mean', 'count']).round(3)
    
    # Create visualization data
    chart_data = pd.DataFrame({
        'Resolution_Type': category_stats.index,
        'Count': category_stats.values
    })
    
    # Create category distribution chart
    resolution_chart = alt.Chart(chart_data).mark_bar().encode(
        x=alt.X('Count:Q', title='Number of Cases'),
        y=alt.Y('Resolution_Type:N', sort='-x', title='Resolution Type'),
        tooltip=['Resolution_Type', 'Count']
    ).properties(
        title='Distribution of Complaint Resolutions',
        width=600,
        height=400
    )
    
    # Create sentiment by category chart
    sentiment_data = merged_df.groupby('resolution_category')['sentiment_score'].mean().reset_index()
    sentiment_chart = alt.Chart(sentiment_data).mark_bar().encode(
        x=alt.X('sentiment_score:Q', title='Average Sentiment Score'),
        y=alt.Y('resolution_category:N', sort='-x', title='Resolution Type'),
        color=alt.Color('sentiment_score:Q', 
                       scale=alt.Scale(scheme='redblue'),
                       title='Sentiment'),
        tooltip=['resolution_category', 
                alt.Tooltip('sentiment_score:Q', format='.3f')]
    ).properties(
        title='Average Sentiment by Resolution Type',
        width=600,
        height=400
    )
    
    # Print statistics
    print("\nResolution Statistics:")
    print(category_stats)
    print("\nPercentage Distribution:")
    print((category_stats / len(merged_df) * 100).round(2))
    print("\nSentiment Analysis by Category:")
    print(sentiment_by_category)
    
    # Return both charts side by side
    combined_chart = alt.hconcat(resolution_chart, sentiment_chart)
    return combined_chart, category_stats, sentiment_by_category

# Run the analysis
combined_chart, stats, sentiment_stats = analyze_resolutions_nlp()
combined_chart

We applied sentiment analysis to understand the emotional tone in resolution descriptions of illegal pet complaints. Using TextBlob, we scored text from -1 (negative) to +1 (positive), with 0 being neutral, to understand how different outcomes are communicated.

Key Findings:

Most Common Resolution: "Warning Issued" (34% of cases)
Most Negative Tone: "Access Denied" (-0.192)
Most Neutral: "Violations Found" (-0.010)
Overall Trend: Slightly negative to neutral tone across most categories

The analysis reveals that while most communications maintain a professional, slightly negative tone, failed inspection attempts ("Access Denied") are communicated more negatively. This suggests a consistent official communication style with variations based on resolution outcome.

Resolution Distribution Analysis:

Major Categories:

"Warning Issued" leads (1,335 cases, 34%)
"No Resolution Recorded" second (825 cases, 21%)
Three categories tie at ~500 cases each (12-14%): "Case Closed," "No Violations Found," "Access Denied"

Key Insight:
The data shows a preference for issuing warnings over finding violations, though a concerning 21% lack recorded resolutions. Access to inspection sites remains a notable challenge.